# Collect PUPPI data

This notebook collects raw PUPPI data and makes observation directories for it. `collect-uppi-data` does the same for GUPPI. Note that the produced observations may end up on the wrong disk (`/data` rather than the permanent-but-slow `/psr_archive`); use `maintain-symlinks.ipynb` to check that out.

In [1]:
%load_ext autoreload

%autoreload 2

In [1]:
import os
from glob import glob
import subprocess
import shutil
import traceback
import random
import pickle
from os.path import join

import joblib
from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import psrchive

import pipe

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [3]:
pipe.process_uppi_dir("/psr_archive/hessels/archibald/0337+17/raw/AO")

INFO:root:Processing uppi batch 56554 0223 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56554 0225 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56584 0590 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56584 0592 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56599 0119 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56599 0121 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56620 0358 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56620 0360 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56645 0559 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56645 0561 in /psr_archive/hessels/archibald/0337+17/raw/AO
INFO:root:Processing uppi batch 56736 0280 in /psr_archive/hessels/archibald/033

In [4]:
!ls /psr_archive/hessels/archibald/0337+17/raw/AO/*.failure

ls: cannot access /psr_archive/hessels/archibald/0337+17/raw/AO/*.failure: No such file or directory


In [23]:
!rm /psr_archive/hessels/archibald/0337+17/raw/AO/*.failure

In [27]:
!ls /psr_archive/hessels/archibald/0337+17/raw/AO/*.success

/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56554_J0337+1715_0223_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56554_J0337+1715_0225_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56584_J0337+1715_0590_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56584_J0337+1715_0592_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56599_J0337+1715_0119_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56599_J0337+1715_0121_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56620_J0337+1715_0358_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56620_J0337+1715_0360_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56645_J0337+1715_0559_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56645_J0337+1715_0561_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/puppi_56736_J0337+1715_0280_0001.f

In [28]:
!rm /psr_archive/hessels/archibald/0337+17/raw/AO/*.success

In [34]:
pipe.process_uppi_dir("/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC")

INFO:root:Processing uppi batch 55989 0068 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 55992 0070 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 55995 0003 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 55995 0004 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 55995 0005 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 55998 0036 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 55998 0037 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 56001 0039 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 56005 0043 in /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC
INFO:root:Processing uppi batch 56008 0047 in /psr_archive/hessels/archibald/0337+17/raw/AO

In [29]:
!ls /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/*.failure

/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_56222_J0337+17_1084_0001.fits.failure


In [30]:
!rm /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/*.failure

In [31]:
!ls /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/*.success

/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_55989_0337+1715_0068_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_55992_0337+1715_0070_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_55995_0337+1715_0003_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_55995_0337+1715_0004_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_55995_0337+1715_0005_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_55998_0337+1715_0036_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_55998_0337+1715_0037_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_56001_0337+1715_0039_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_56005_J0337+17_0043_0001.fits.success
/psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/puppi_56008_J0337+17_0047_0001.fits.success
/p

In [32]:
!rm /psr_archive/hessels/archibald/0337+17/raw/AO/from_UBC/*.success